### <b> Apple Classifier </b> 

This notebook is a Deep Learning project  to train and deploy an image classifier using FastAI's API.

In [1]:
from fastbook import *
from fastai.vision.widgets import *

##### 1. Gathering Data from the web

In [2]:
apple_types = ['red', 'green', 'logo']
path = Path('apple')

In [ ]:
# Search for and download images
for a in apple_types:
    dest = (path/a)
    results = search_images_ddg(f'{a} apple images')
    for i in range(len(results)):
        try:
            download_url(results[i], dest / f'{i}.jpg')
        except Exception as e:
            print(f"Error downloading {results[i]}: {e}")

In [4]:
#Check download images to designated path
images = get_image_files(path)
images

(#580) [Path('apple/green/0.jpg'),Path('apple/green/1.jpg'),Path('apple/green/10.jpg'),Path('apple/green/100.jpg'),Path('apple/green/101.jpg'),Path('apple/green/102.jpg'),Path('apple/green/103.jpg'),Path('apple/green/104.jpg'),Path('apple/green/105.jpg'),Path('apple/green/106.jpg')...]

In [8]:
# Check for corrupt images
failed = verify_images(images)
failed

(#25) [Path('apple/green/108.jpg'),Path('apple/green/117.jpg'),Path('apple/green/133.jpg'),Path('apple/green/174.jpg'),Path('apple/green/197.jpg'),Path('apple/green/199.jpg'),Path('apple/green/38.jpg'),Path('apple/green/51.jpg'),Path('apple/logo/104.jpg'),Path('apple/logo/11.jpg')...]

In [9]:
#Remove corrupt images
failed.map(Path.unlink)

(#25) [None,None,None,None,None,None,None,None,None,None...]